# HPPC Parameter Extraction (Full)

The logic in this notebook is almost identical to that of `extract_hppc_partial.py`. 



In [1]:


%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
from data_tools.query import DBClient 
from data_tools.collections import TimeSeries
import datetime

from dotenv import load_dotenv

load_dotenv()

ERROR:data_tools.query.postgresql_query:Could not find a POSTGRESQL_USERNAME in .env!
ERROR:data_tools.query.postgresql_query:Could not find a POSTGRESQL_PASSWORD in .env!
ERROR:data_tools.query.postgresql_query:Could not find a POSTGRESQL_DATABASE in .env!
ERROR:data_tools.query.postgresql_query:Could not find a POSTGRESQL_ADDRESS in .env!


True

In [36]:
client = DBClient(influxdb_token="s4Z9_S6_O09kDzYn1KZcs7LVoCA2cVK9_ObY44vR4xMh-wYLSWBkypS0S0ZHQgBvEV2A5LgvQ1IKr8byHes2LA==", influxdb_org="8a0b66d77a331e96")

start = datetime.datetime.fromisoformat("2025-04-11T00:00:00Z")
stop = datetime.datetime.fromisoformat("2025-04-14T00:00:00Z")

pack_voltage = client.query_time_series(start=start, stop=stop, field="TotalPackVoltage", bucket="CAN_log", granularity=0.1, units="V", car="Brightside")
pack_current = client.query_time_series(start=start, stop=stop, field="PackCurrent", bucket="CAN_log", granularity=0.1, units="A", car="Brightside")

pack_current_aligned, pack_voltage_aligned = TimeSeries.align(pack_current, pack_voltage)

Creating client with API Token: s4Z9_S6_O09kDzYn1KZcs7LVoCA2cVK9_ObY44vR4xMh-wYLSWBkypS0S0ZHQgBvEV2A5LgvQ1IKr8byHes2LA==
Creating client with Org: 8a0b66d77a331e96


In [9]:
fig, ax = plt.subplots()

ax2 = ax.twinx()
# ax.plot(pack_current.datetime_x_axis, pack_current)
ax2.plot(pack_voltage.datetime_x_axis, pack_voltage)
ax2.set_title("Pack Voltage during HPPC test")
ax2.set_ylabel("Pack Voltage (V)")
ax2.set_xlabel("Time (?)")
plt.tight_layout()
plt.show()

<IPython.core.display.Javascript object>

# Offline Parameter Identification 

For each pulse, we need to manually identify when the zero-state reponse (behaviour due to capacitor) is occuring.

In [10]:
pulse_indices = [421210, 379410, 329816, 284051, 238416, 192883, 148060, 100789, 56830, 8274]
beginning_indices = [11024, 57700, 102089, 148200, 193033, 238456, 284061, 329916, 380010, 394210]
ending_indices = [11274, 57900, 102489, 148460, 193283, 238716, 284261, 330146, 380110, 405210]

In [12]:
fig.clear()
fig, ax = plt.subplots()
ax2 = ax.twinx()

ax.plot(pack_voltage)
plt.show()

<IPython.core.display.Javascript object>

## Pulse Examination

In [13]:
fig.clear()
fig, ax = plt.subplots()
ax2 = ax.twinx()

beginning_index = 193000
end_index = 194000

pack_voltage_extracted = pack_voltage[beginning_index:end_index]
pack_current_extracted = pack_current[beginning_index:end_index]

A_index = 10
B_index = 12
C_index = 200
D_index = 202
E_index = 700

ax.plot(pack_voltage_extracted)
ax.axvline(x=A_index, color='tab:red')
ax.axvline(x=B_index, color='tab:orange')
ax.axvline(x=C_index, color='tab:green')
ax.axvline(x=D_index, color='tab:blue')
ax.axvline(x=E_index, color='red')

plt.show()

<IPython.core.display.Javascript object>

In [14]:
fig.clear()
fig, ax = plt.subplots()

ax.plot(pack_current_extracted)
ax.axvline(x=A_index, color='tab:red')
ax.axvline(x=B_index, color='tab:orange')
ax.axvline(x=C_index, color='tab:green')
ax.axvline(x=D_index, color='tab:blue')
ax.axvline(x=E_index, color='red')

plt.show()

<IPython.core.display.Javascript object>

What we are basically doing here is finding the resistance by looking at the voltage DIFFERENCE compared to the current DIFFERENCE from going from 0A to 40A and 40A back down to 0A. That's why we are taking the last and first element of the current, respectively (what happens between isn't a concern to us).

In [15]:
U_A = pack_voltage_extracted[A_index]
U_B = pack_voltage_extracted[B_index]
U_C = pack_voltage_extracted[C_index]
U_D = pack_voltage_extracted[D_index]

I_AB = pack_current_extracted[A_index:B_index]
R_0_AB = (U_A - U_B) / I_AB[-1]

I_CD = pack_current_extracted[C_index:D_index]
R_0_DC = (U_D - U_C) / I_CD[0]

R_0 = (R_0_AB + R_0_DC) / 2
U_oc = U_A

print(f"R_0: {R_0:.4f} Ohms")
print(f"U_oc: {U_oc:.1f} V")

R_0: -0.0001 Ohms
U_oc: 118.4 V


In [16]:
def BC_func(t, _U_oc, _I, _R_0, _R_P1, _C_P1, _U_discharge):
    _a = _U_oc - _I * _R_0 - _U_discharge
    _b = _I * _R_P1 
    _c = _R_P1 * _C_P1
    # _d = _I * _R_P2 
    # _e = _R_P2 * _C_P2
    return _a - _b * (1 - np.exp(-t / _c))

def DE_func(t, _U_oc, _R_0, _R_P1, _C_P1, _U_P1, _U_discharge):
    _a = _U_oc - _U_discharge
    _b = _U_P1
    _c = _R_P1 * _C_P1
    # _d = _U_P2
    # _e = _R_P2 * _C_P2
    return _a - _b * np.exp(-t / _c)

In [17]:
def objective(t, _R_0, _R_P1, _C_P1c, _U_P1, _U_discharge):
    _U_oc = pack_voltage_extracted[0]
    _I = pack_current_extracted
    initial_time_BC = _I.x_axis[B_index]
    initial_time_DE = _I.x_axis[D_index]
    discharge_time = _I.x_axis[C_index] - _I.x_axis[B_index]
    voltage = []
    next_voltage = 42
    
    t = t.astype(int)
    
    for i in t:
        if i <= A_index:
            next_voltage = _U_oc
        
        if A_index < i <= B_index:
            next_voltage = _U_oc - _I[i] * _R_0
        
        if B_index < i <= C_index:
            _t = _I.x_axis[i] - initial_time_BC
            time_factor = (_I.x_axis[i] - initial_time_BC) / discharge_time
            discharged_voltage = time_factor * _U_discharge

            next_voltage = BC_func(_t, _U_oc, _I[i], _R_0, _R_P1, _C_P1c, discharged_voltage)
        
        if C_index < i <= D_index:
            next_voltage = _U_oc - _U_P1 - _U_discharge - _I[i] * _R_0
        
        if D_index < i <= E_index:
            _t = _I.x_axis[i] - initial_time_DE
            next_voltage = DE_func(_t, _U_oc, _R_0, _R_P1, _C_P1c, _U_P1, _U_discharge)
            
        voltage.append(next_voltage)
        
    return voltage

In [18]:
from scipy import optimize 
from sklearn.preprocessing import MinMaxScaler


# Original (non-normalized) bounds
lower_bounds = np.array([
    0.01,  # R_0
    0.01,  # R_P1
    0.0,  # C_P1c
    # 0.01,  # R_P2
    # 0.0,  # C_P2c
    0.01,  # U_P1
    # 0.01,  # U_P2
    0.01   # U_discharged_data
])
upper_bounds = np.array([
    0.5,    # R_0
    0.5,    # R_P1
    2000.0, # C_P1c
    # 0.5,    # R_P2
    # 2000.0, # C_P2c
    15.0,   # U_P1
    # 15.0,   # U_P2
    15.0    # U_discharged_data
])

bounds = list(zip(lower_bounds, upper_bounds))

# Create a scaler to map [lower_bounds, upper_bounds] <-> [0, 1]
scaler = MinMaxScaler()
scaler.fit(np.vstack([lower_bounds, upper_bounds]))  # shape (2, n_features)
scaled_bounds = scaler.transform([lower_bounds, upper_bounds])
scaled_lower_bounds = scaled_bounds[0]
scaled_upper_bounds = scaled_bounds[1]

# --- Scaled objective function ---
def scaled_objective(t, *scaled_params):
    real_params = scaler.inverse_transform([scaled_params])[0]
    return objective(t, *real_params)

# Idk, seems okay to me
# p0 = [R_0, R_0, 300., R_0, 300., 1.0, 1.0, 0.3]
p0 = [R_0, R_0, 300., 1.0, 0.3]
# --- Define initial guess and scale it ---
p0_scaled = scaler.transform([p0])[0]

y_data = pack_voltage_extracted[0:E_index + 1]
x_data = np.arange(len(y_data))

objective(x_data, *p0)

x_scaled, pcov = optimize.curve_fit(
    scaled_objective, 
    x_data, 
    y_data, 
    p0=p0_scaled, 
    maxfev=4000,
    bounds=(scaled_lower_bounds, scaled_upper_bounds),
)
x = scaler.inverse_transform([x_scaled])[0]

fig, ax = plt.subplots()

ax.plot(objective(x_data, *x), label="Predicted")
ax.plot(pack_voltage_extracted, label="Measured")
ax.set_title("Fitting ECM to HPPC pulse data")
ax.set_xlabel("Time (10ms)")
ax.set_ylabel("Voltage (V)")
plt.legend()
plt.show()

/var/folders/26/b439y_h53ls_5yw6crm1x55r0000gn/T/ipykernel_40403/893259988.py:15: RuntimeWarning: overflow encountered in exp
  return _a - _b * np.exp(-t / _c)


ValueError: `x0` is infeasible.

In [12]:
[print(f"{_x:.4f}") for _x in x]
print(U_oc)
print(beginning_index)

0.5000
0.5000
85.6304
2.7977
2.7812
95.17948718161921
394200


In [19]:
# R_0_data = [0.1807, 0.1413, 0.1254, 0.1146, 0.1056, 0.1068]
# R_P_data = [0.1935, 0.1574, 0.1983, 0.1651, 0.1931, 0.1459]
# C_P_data = [197.8862, 288.7784, 246.6670, 326.9496, 269.9395, 348.3759]
# U_oc_data = [131.6941, 129.4818, 125.8064, 122.0371, 118.3019, 115.5432]
R_0_data = [0.1879, 0.1451, 0.1337, 0.1167, 0.1107, 0.1108, 0.1138, 0.1383, 0.1834, 0.1834]
R_P_data = [0.0362, 0.0259, 0.0283, 0.0319, 0.0549, 0.0406, 0.0448, 0.0851, 0.2445, 0.2445]
C_P_data = [436.3630, 508.6304, 577.8932, 528.8127, 496.0647, 567.4015, 539.2374, 405.4668, 12.6725, 12.6725]
U_oc_data = [131.82069619200664, 129.6132478661512, 126.0299145327376, 122.0961538488888, 118.42117413364828, 115.6752136778048, 112.0113881011542, 108.10968770035525, 97.38888889107041, 95.17948718161921]
indices = [11000, 57700, 102000, 148200, 193000, 238400, 284000, 329900, 380000, 394200]

In [20]:
from scipy import integrate

discharged_capacity = np.cumsum(pack_current * pack_current.granularity)
total_discharged_capacity = integrate.simpson(y=pack_current, x=pack_current.x_axis)

In [21]:
initial_soc = 0.94
final_soc = 0.03

In [22]:
true_charge_capacity = total_discharged_capacity / (initial_soc - final_soc)
true_charge_capacity

np.float64(151144.3076430995)

In [23]:
# plt.close()
fig, ax = plt.subplots()

ax.plot(discharged_capacity)
plt.show()

<IPython.core.display.Javascript object>

In [30]:
initial_soc = 1 + discharged_capacity[9000] / total_discharged_capacity
soc = initial_soc - (discharged_capacity / total_discharged_capacity)
SOC_indices = soc[indices]

# plt.close()
fig, ax = plt.subplots()

ax.plot(soc)
plt.show()

<IPython.core.display.Javascript object>

In [31]:
raw_params = f"""
R_0_data = {[float(x) for x in R_0_data]}
R_P_data = {[float(x) for x in R_P_data]}
C_P_data = {[float(x) for x in C_P_data]}
Uoc_data = {[float(x) for x in U_oc_data]}
SOC_data = {[float(x) for x in SOC_indices]}
"""

print(raw_params)


R_0_data = [0.1879, 0.1451, 0.1337, 0.1167, 0.1107, 0.1108, 0.1138, 0.1383, 0.1834, 0.1834]
R_P_data = [0.0362, 0.0259, 0.0283, 0.0319, 0.0549, 0.0406, 0.0448, 0.0851, 0.2445, 0.2445]
C_P_data = [436.363, 508.6304, 577.8932, 528.8127, 496.0647, 567.4015, 539.2374, 405.4668, 12.6725, 12.6725]
Uoc_data = [131.82069619200664, 129.6132478661512, 126.0299145327376, 122.0961538488888, 118.42117413364828, 115.6752136778048, 112.0113881011542, 108.10968770035525, 97.38888889107041, 95.17948718161921]
SOC_data = [1.0000113625036742, 0.8815270004406436, 0.7671930370357474, 0.6206091141019272, 0.4911640598651529, 0.3606345466159997, 0.2368791855068657, 0.12073810977228183, 0.014565800462693401, 0.0070701434475712865]



In [32]:
print(f"Charge Capacity: {total_discharged_capacity}")

Charge Capacity: 137541.31995522053


In [78]:
%load_ext autoreload
%autoreload 2

In [ ]:
import physics


In [84]:
%aimport physics.models.battery

battery_model = physics.models.battery.BatteryModel(battery_config=physics.models.battery.BatteryModelConfig(
    R_0_data=R_0_data,
    R_P_data=R_P_data,
    C_P_data=C_P_data,
    Uoc_data=U_oc_data,
    SOC_data=SOC_indices,
    max_current_capacity=42.9,
    max_energy_capacity=500,
    Q_total=total_discharged_capacity
), state_of_charge=0.99)

pack_power = -pack_current_aligned * pack_voltage_aligned
predicted_soc, predicted_voltage = battery_model.update_array(pack_power, pack_power.granularity)

In [85]:
dir(battery_model)

['C_P',
 'C_P_coefficients',
 'R_0',
 'R_0_coefficients',
 'R_P',
 'R_P_coefficients',
 'U_L',
 'U_P',
 'U_oc',
 'U_oc_coefficients',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_evolve',
 '_update_array_py',
 'charge_current',
 'discharge_current',
 'max_current_capacity',
 'max_energy_capacity',
 'nominal_charge_capacity',
 'state_of_charge',
 'tau',
 'update_array']

In [65]:
dir(battery_model)

['C_P',
 'C_P_coefficients',
 'R_0',
 'R_0_coefficients',
 'R_P',
 'R_P_coefficients',
 'U_L',
 'U_P',
 'U_oc',
 'U_oc_coefficients',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_evolve',
 '_update_array_py',
 'charge_current',
 'discharge_current',
 'max_current_capacity',
 'max_energy_capacity',
 'nominal_charge_capacity',
 'state_of_charge',
 'tau',
 'update_array']

In [53]:
plt.close()
fig, ax = plt.subplots()


ax.plot(predicted_voltage, label="Predicted")
ax.plot(pack_voltage_aligned, label="Measured")
plt.legend()
plt.show()

<IPython.core.display.Javascript object>

In [45]:
plt.close()
fig, ax = plt.subplots()


ax.plot(predicted_voltage, label="Predicted")
ax.plot(pack_voltage_aligned, label="Measured")
plt.legend()
plt.show()

<IPython.core.display.Javascript object>